In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as _sum

In [2]:
conf = SparkConf()
conf.set("spark.hadoop.fs.s3a.impl",
                 "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.access.key", "admin")
conf.set("spark.hadoop.fs.s3a.secret.key", "123456789")
conf.set("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider',
            'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set(
    "spark.jars.packages",
    "io.delta:delta-core_2.12:1.0.0,"
    "org.apache.hadoop:hadoop-aws:3.1.1,"
    "com.amazonaws:aws-java-sdk:1.11.271,"
    "com.amazonaws:aws-java-sdk-bundle:1.11.271,"
    "software.amazon.awssdk:url-connection-client:2.15.40",
)
conf.set("spark.sql.extensions",
            "io.delta.sql.DeltaSparkSessionExtension")
conf.set(
    "spark.sql.catalog.spark_catalog",
    "org.apache.spark.sql.delta.catalog.DeltaCatalog",
)
conf.set("spark.databricks.delta.merge.repartitionBeforeWrite.enabled","true")

spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .master("local[*]")
    .getOrCreate()
)

23/03/27 17:00:06 WARN Utils: Your hostname, ducdn-G3-3579 resolves to a loopback address: 127.0.1.1; using 10.1.124.58 instead (on interface enp3s0)
23/03/27 17:00:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ducdn/Documents/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ducdn/.ivy2/cache
The jars for the packages stored in: /home/ducdn/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c3715b8-a97e-4ead-a2e3-3fba1776d572;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.hadoop#hadoop-aws;3.1.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central

### Bronzen

In [10]:
orderDF = spark.read.format("delta").load("s3a://datalake/sliver/aggreateTable")
orderDF.show()

+------------+----------+--------------------+---------+------------------+--------+----------+-------------+-----------+
|order_number|order_time|               email|purchaser|      product_name|quantity|unit_price|    timestamp|total_price|
+------------+----------+--------------------+---------+------------------+--------+----------+-------------+-----------+
|       10078|     19037|  gbailey@foobar.com|     1002|12-pack drill bits|      46|       0.8|1679911097439|       36.8|
|       10090|     19037|  gbailey@foobar.com|     1002|12-pack drill bits|      40|       0.8|1679911157565|       32.0|
|       10039|     19037|sally.thomas@acme...|     1001|12-pack drill bits|      45|       0.8|1679909632361|       36.0|
|       10064|     19037|sally.thomas@acme...|     1001|12-pack drill bits|      36|       0.8|1679911027297|  28.800001|
|       10119|     19037|sally.thomas@acme...|     1001|12-pack drill bits|      19|       0.8|1679911302860|       15.2|
|       10130|     19037

In [5]:
productDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.products")
productDF.orderBy("id").show()

+---+-------------+---+------------------+--------------------+------+---------+
| op|        ts_ms| id|              name|         description|weight|before_id|
+---+-------------+---+------------------+--------------------+------+---------+
|  r|1679384739684|101|           scooter|Small 2-wheel sco...|  3.14|      101|
|  r|1679384739684|102|       car battery|     12V car battery|   8.1|      102|
|  r|1679384739684|103|12-pack drill bits|12-pack of drill ...|   0.8|      103|
|  r|1679384739684|104|            hammer|12oz carpenter's ...|  0.75|      104|
|  r|1679384739685|105|            hammer|14oz carpenter's ...| 0.875|      105|
|  r|1679384739685|106|            hammer|16oz carpenter's ...|   1.0|      106|
|  r|1679384739685|107|             rocks|box of assorted r...|   5.3|      107|
|  r|1679384739685|108|            jacket|water resistent b...|   0.1|      108|
|  r|1679384739686|109|        spare tire|  24 inch spare tire|  22.2|      109|
+---+-------------+---+-----

In [6]:
customerDF = spark.read.format("delta").load("s3a://datalake/brozen/cdc.inventory.customers")
customerDF.show()

+---+-------------+----+----------+---------+--------------------+---------+
| op|        ts_ms|  id|first_name|last_name|               email|before_id|
+---+-------------+----+----------+---------+--------------------+---------+
|  r|1679384739666|1001|     Sally|   Thomas|sally.thomas@acme...|     1001|
|  r|1679384739667|1004|      Anne|Kretchmar|  annek@noanswer.org|     1004|
|  r|1679384739666|1002|    George|   Bailey|  gbailey@foobar.com|     1002|
|  r|1679384739667|1003|    Edward|   Walker|       ed@walker.com|     1003|
+---+-------------+----+----------+---------+--------------------+---------+



### Sliver

In [12]:
orderDF = spark.read.format("delta").load("s3a://datalake/sliver/agg")
orderDF.show(truncate=False)

+---------------------+-----------------------+--------------+
|email                |timestamp              |total_quantity|
+---------------------+-----------------------+--------------+
|sally.thomas@acme.com|2023-03-21 22:09:43.112|200           |
|sally.thomas@acme.com|2023-03-21 22:02:31.555|168           |
|sally.thomas@acme.com|2023-03-21 20:58:47.407|1             |
|sally.thomas@acme.com|2023-03-21 21:53:38.644|43            |
|gbailey@foobar.com   |2023-03-21 22:09:38.103|200           |
|gbailey@foobar.com   |2023-03-21 22:02:41.576|360           |
|gbailey@foobar.com   |2023-03-21 20:58:47.408|4             |
|annek@noanswer.org   |2023-03-21 21:53:43.644|26            |
|annek@noanswer.org   |2023-03-21 21:53:48.652|38            |
|ed@walker.com        |2023-03-21 22:09:48.125|148           |
|ed@walker.com        |2023-03-21 20:58:47.408|1             |
|sally.thomas@acme.com|2023-03-21 22:02:26.546|224           |
|ed@walker.com        |2023-03-21 22:02:36.567|328     

In [ ]:
path_table = "s3a://datalake/brozen/cdc.inventory.orders"
df = spark.read.format("delta").option("versionAsOf", 0).load(path_table)
df.show()

In [ ]:
delta_table = DeltaTable.forPath(spark, "s3a://datalake/brozen/cdc.inventory.orders")
history = delta_table.history()
history.show()

In [ ]:
df = spark.read.format("delta").load("s3a://datalake/sliver")
df.show()